<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #10: Twitter
`Fecha de entrega: Noviembre 14, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

(Si exponen sus claves, les haré maldades 😈)

❌🎃❌ ¡No Halloween!

In [1]:
import pandas as pd 
import tweepy

In [2]:
#leer llaves
keys = pd.read_csv('D:/Users/User/Imágenes/Especializacion Analitica Estrategica de Datos/Semestre II/Electiva NLP/clase_12/keys_demo.txt', header=None)
keys = dict(zip(keys[0],keys[1]))

In [3]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [5]:
tweets = tweepy.Cursor(api.search,
                       q=['partido -RT'],
                       lang='es',
                       geocode=f'4.649178,-74.062827,45mi',
                       tweet_mode='extended',
                       include_rts=False).items(100)

In [7]:
t=[]
for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogotá")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)

In [8]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,Chile le debía un buen partido a su técnico. h...,2020-11-14 03:18:10,Bogotá,Bogotá,1327450727170629632,https://twitter.com/i/web/status/1327450727170...
1,Esto fue lo que le faltó al partido hoy. https...,2020-11-14 03:11:10,Bogotá,"Bogotá, D.C., Colombia",1327448963625218050,https://twitter.com/i/web/status/1327448963625...
2,@JosePeseiro @FVF_Oficial Botaste el partido a...,2020-11-14 03:10:44,Bogotá,"Bogotá, Colombia",1327448854711705600,https://twitter.com/i/web/status/1327448854711...
3,"#Video El partido estaba apretado, pero el ven...",2020-11-14 03:10:38,Bogotá,"Bogotá, Colombia",1327448828396572672,https://twitter.com/i/web/status/1327448828396...
4,@yosoyelcarlos1 Claro que tiene que ver ... Va...,2020-11-14 03:09:42,Bogotá,"Bogota, Colombia",1327448595436630016,https://twitter.com/i/web/status/1327448595436...


In [9]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,Chile le debía un buen partido a su técnico. h...,2020-11-14 03:18:10,Bogotá
1,Esto fue lo que le faltó al partido hoy. https...,2020-11-14 03:11:10,Bogotá
2,@JosePeseiro @FVF_Oficial Botaste el partido a...,2020-11-14 03:10:44,Bogotá
3,"#Video El partido estaba apretado, pero el ven...",2020-11-14 03:10:38,Bogotá
4,@yosoyelcarlos1 Claro que tiene que ver ... Va...,2020-11-14 03:09:42,Bogotá


In [10]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [11]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,city,clean,links,hashtags,mentions
0,Chile le debía un buen partido a su técnico. h...,2020-11-14 03:18:10,Bogotá,Chile le debía un buen partido a su técnico.,[https://t.co/z2ccywVI0q],[],[]
1,Esto fue lo que le faltó al partido hoy. https...,2020-11-14 03:11:10,Bogotá,Esto fue lo que le faltó al partido hoy.,[https://t.co/OXUI45aL85],[],[]
2,@JosePeseiro @FVF_Oficial Botaste el partido a...,2020-11-14 03:10:44,Bogotá,Botaste el partido ante Brasil sacando a Sot...,[],[],"[@JosePeseiro, @FVF_Oficial]"
3,"#Video El partido estaba apretado, pero el ven...",2020-11-14 03:10:38,Bogotá,"El partido estaba apretado, pero el venezolan...",[https://t.co/rvhP0v1dD2],"[#Video, #EmociónEnGrande]",[]
4,@yosoyelcarlos1 Claro que tiene que ver ... Va...,2020-11-14 03:09:42,Bogotá,Claro que tiene que ver ... Vamos dos partido...,[],[#Barranquilla],[@yosoyelcarlos1]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

❌🎃❌ ¡No Halloween!

In [12]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_sp = stopwords.words('spanish')

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto

tfidf_vect = TfidfVectorizer(preprocessor=pre_procesado, ngram_range=(1,1), max_features=100)
tfidf = tfidf_vect.fit_transform(data.clean.values)


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

voc = [k for k,v in sorted(tfidf_vect.vocabulary_.items(), key=lambda kv: kv[1])]
temp = pd.DataFrame(tfidf.toarray())
temp = cosine_similarity(temp.T.values)
temp = pd.DataFrame(temp)
temp.columns = voc
temp.index = voc
temp

,acuerdo,acá,además,ahora,ahí,así,aunque,bajo,barranquilla,bien,...,tan,tarde,tercera,técnico,uruguay,vamos,venezuela,veo,ver,vi
acuerdo,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.238932,0.193159,0.000000,0.000000,0.689879,0.233932,0.0
acá,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.358802,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
además,0.000000,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.605331,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.093267,0.087169,0.000000,0.000000,0.0
ahora,0.000000,0.0,0.000000,1.0,0.0,0.274824,0.0,0.0,0.263830,0.000000,...,0.000000,0.000000,0.211812,0.000000,0.101570,0.000000,0.000000,0.000000,0.000000,0.0
ahí,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.178418,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vamos,0.000000,0.0,0.093267,0.0,0.0,0.000000,0.0,0.0,0.000000,0.056458,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.241086,0.000000,0.333230,0.0
venezuela,0.000000,0.0,0.087169,0.0,0.0,0.000000,0.0,0.0,0.000000,0.052766,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.241086,1.000000,0.000000,0.115061,0.0
veo,0.689879,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.364827,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.209764,0.000000,0.000000,1.000000,0.254042,0.0
ver,0.233932,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.221657,0.000000,0.000000,0.071129,0.333230,0.115061,0.254042,1.000000,0.0


In [15]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [18]:
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
0,2020-11-14,99


In [19]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})

trace2 = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values+10,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días Rosados',
                   line={'color':'pink'})

layout = go.Layout(title='Número de tuits por día')

fig = go.Figure(data=[trace, trace2], layout=layout)
iplot(fig)

In [16]:
trace = go.Heatmap(z=temp.values.tolist(),
                   x=temp.index.values,
                   y=temp.columns.values,
                   colorscale='Oranges')

layout = go.Layout(title='🔥 Mapa de calor entre palabras 🔥',
                   width=800, height=800)

fig = go.Figure(data=[trace],layout=layout)
iplot(fig)